In [1]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import GEOparse
import cobra
import riptide

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

### Read transcriptomics file

In [20]:
# Download from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE57945
fname = 'data/GSE57945_family.soft.gz'
folder_name = 'data/GSE57945_RAW/'

# Read GEOparse
gse = GEOparse.get_GEO(filepath=fname, silent=True)

# Get all diagnosis
diagnosis_data = pd.DataFrame()

for key, value in gse.gsms.items():
    gsm = key
    name = value.metadata['description'][0]
    diagnosis = value.metadata['characteristics_ch1'][4].split(':')[-1].strip()
    
    temp = pd.DataFrame([[gsm, name, diagnosis]])
    diagnosis_data = diagnosis_data.append(temp)
    
diagnosis_data = diagnosis_data.reset_index(drop=True)
diagnosis_data.columns = ['gsm', 'name', 'diagnosis']

In [100]:
dfs = {
    'CD': pd.DataFrame(),
    'UC': pd.DataFrame(),
    'Not IBD': pd.DataFrame(),
    'not IBD': pd.DataFrame()
}

for index, row in diagnosis_data.iterrows():
    tmp = pd.read_csv(folder_name + row['gsm'] + '_' + row['name'] + '.txt', sep='\t')
    
    if len(dfs[row['diagnosis']]) == 0:
        tmp = tmp[['Gene ID', 'Gene Symbol', row['name']]]
        dfs[row['diagnosis']] = tmp
    else:
        tmp = tmp[['Gene ID', row['name']]]
        dfs[row['diagnosis']] = pd.merge(dfs[row['diagnosis']], tmp, on='Gene ID')
        
# Combining not IBD and Not IBD        
dfs['Not IBD'] = pd.merge(dfs['Not IBD'], dfs['not IBD'], on=['Gene ID', 'Gene Symbol'])     
del dfs['not IBD']


# Gene Map with all the genes present in Recon3D
gene_name_num = pd.read_csv('data/gene_name_number.tsv', sep='\t')
gene_name_num = gene_name_num[['gene_number', 'symbol']]

id_map = {}
for keys in dfs.keys():
    # Generate ID map
    id_map[keys] = diagnosis_data.loc[diagnosis_data['diagnosis']==keys, 'name'].tolist()
    
    # merge gene number annotations to dataframe
    dfs[keys] = pd.merge(gene_name_num, dfs[keys], left_on='symbol', right_on='Gene Symbol')
    
id_map['Not IBD'] += diagnosis_data.loc[diagnosis_data['diagnosis']=='not IBD', 'name'].tolist()    

### Run Riptide for all patients

In [ ]:
# Load Model
model = cobra.io.load_matlab_model('data/Recon3D_301.mat')
cobra.util.array.create_stoichiometric_matrix(model).shape

def run_riptide(fname, pat_name):
    
    transcript_abundances = riptide.read_transcription_file(fname, replicates=False, norm=False)
    riptide_object = riptide.contextualize(model=model, transcriptome=transcript_abundances, fraction=1.)

    riptide_fva = riptide_object.flux_samples
    riptide_fva['sample_number'] = pat_name
    
    return riptide_fva

In [110]:
for keys in dfs.keys():
    print('Running for - {}'.format(keys))
    for pat_id in id_map[keys]:
        dfs['CD'][['gene_number', pat_id]].to_csv('sample.tsv', header=False, index=False, sep='\t')
        pat_output = run_riptide('sample.tsv', pat_id)
        pat_output.to_csv('output/{}.csv'.format(pat_id), index=False)
        
# Combine flux samples  
for keys in dfs.keys():
    list_of_pat_flux = []
    for pat_id in id_map[keys]:
        list_of_pat_flux.append(pd.read_csv('output/{}.csv'.format(pat_id)))
        break
    pd.concat(list_of_pat_flux, join='inner', ignore_index=0).to_csv(\
    'output/riptide_{}_flux_sample.csv'.format(keys), index=False)        

Running for - CD

Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 817 from 13543 (93.97% change)
Metabolites pruned to 773 from 8399 (90.8% change)
Flux through the objective DECREASED to ~991.2 from ~1000.0 (0.88% change)
Context-specific metabolism correlates with transcriptome (r=-0.191, p<0.001 *)

RIPTiDe completed in, 5 minutes and 51 seconds 



### Run ML model on aggregated flux files

In [125]:
df_cd = pd.read_csv('output/riptide_CD_flux_sample.csv')
df_cd = pd.read_csv('output/riptide_Not IBD_flux_sample.csv')

# Using only iCD patients
cd_ibd = pd.read_csv('data/2-7-2021 cd_uc_split.csv')
df_cd = df_cd.loc[df_cd['sample_number'].isin(cd_ibd.loc[cd_ibd['value']=='iCD', 'name'].tolist())].reset_index(drop=True)

df_cd['label'] = 1
df_not_ibd['label'] = 0

df = pd.concat([df_cd, df_not_ibd], join='inner') 

##### Model instructions
 - Run model multiple times with different train-val split 
 - Only keeping top reactions from the runs where gains are greater than 0.2. This rule can be modified based on project goals.

In [134]:
imp_list = []
accuracy_tracker = []
f1_tracker = []
precision_tracker = []
recall_tracker = []

val_distribution_tracker = []
imp_df = []

# Run model 100 times
for _ in tqdm(range(100)):

    # Keeping train-val split ratio 80-20
    test_sample = random.sample(list(df_cd['sample_number'].unique()), 10) +\
    random.sample(list(df_not_ibd['sample_number'].unique()), 8)

    # Undersample CD patients to balance the split
    train_sample = list(set(list(random.sample(list(df_cd['sample_number'].unique()), 50)) +\
                            list(df_not_ibd['sample_number'].unique())) - set(test_sample))
    
    df = pd.concat([df_cd, df_not_ibd], join='inner') 
    df = df.reset_index(drop=True)
    
    X_train, y_train = df.loc[df['sample_number'].isin(train_sample), list(set(df.columns)-set(['sample_number', 'label']))].reset_index(drop=True),\
    df.loc[df['sample_number'].isin(train_sample), ['label']].reset_index(drop=True)

    X_test, y_test = df.loc[df['sample_number'].isin(test_sample), list(set(df.columns)-set(['sample_number', 'label']))].reset_index(drop=True),\
    df.loc[df['sample_number'].isin(test_sample), ['label']].reset_index(drop=True)

    y_train = y_train['label']
    y_test = y_test['label']

    rf = RandomForestClassifier(n_estimators=100)
    rf.fit(X_train, y_train)

    y_train_pred = rf.predict(X_train)
    print('Training accuracy on selected features: %.3f' % accuracy_score(y_train, y_train_pred))    

    y_test_pred = rf.predict(X_test)    
    print('Testing accuracy on selected features: %.3f' % accuracy_score(y_test, y_test_pred))    
    print('F1 Score: {}'.format(f1_score(y_test, y_test_pred)))
    print('Distribution: ', max(y_test.mean(), (1-y_test.mean())))
    
    selected_cols = X_train.columns[np.argsort(rf.feature_importances_)[::-1]]
    
    val_distribution_tracker.append(y_test.mean())
    f1_tracker.append(f1_score(y_test, y_test_pred))
    precision_tracker.append(precision_score(y_test, y_test_pred))    
    recall_tracker.append(recall_score(y_test, y_test_pred))        
    accuracy_tracker.append(accuracy_score(y_test, y_test_pred))
    gain = accuracy_score(y_test, y_test_pred)-max(y_test.mean(), (1-y_test.mean()))
    
    if gain>0.2:
        imp_list += list(selected_cols[:50])
        imp_df.append(pd.DataFrame({'reactions': selected_cols[:50], 'rank': range(1, 51)}))
        print('Imp features: ', selected_cols[:10])
        print('-'*20)
        print('Gain: ', gain)
        print('-'*20)
    else:
        print('Failed to gain: ', gain)
        
    print('*'*20)
    print('*'*20)    

  1%|          | 1/100 [00:06<10:34,  6.41s/it]

Training accuracy on selected features: 1.000
Testing accuracy on selected features: 0.907
F1 Score: 0.9100410600970511
Distribution:  0.5246723207401697
Imp features:  Index(['ATP2ter', 'HMGCOAtm', 'EX_uri[e]', 'LPS2', 'FACOAL245_2',
       'sink_lnlncacoa[c]', 'URIt', 'r1466', 'EX_Rtotal[e]', 'MEV_Rt'],
      dtype='object')
--------------------
Gain:  0.3824209714726291
--------------------
********************
********************


  2%|▏         | 2/100 [00:11<09:39,  5.91s/it]

Training accuracy on selected features: 1.000
Testing accuracy on selected features: 0.816
F1 Score: 0.7967618236046017
Distribution:  0.5374228395061729
Imp features:  Index(['ATP2ter', 'PEt', 'EX_pe_hs[e]', 'sink_lnlncacoa[c]', 'r1004', 'r1466',
       'r0431', 'r0430', 'EX_uri[e]', 'FACOAL245_2'],
      dtype='object')
--------------------
Gain:  0.27854938271604934
--------------------
********************
********************


  3%|▎         | 3/100 [00:18<09:50,  6.09s/it]

...

Training accuracy on selected features: 1.000
Testing accuracy on selected features: 0.635
F1 Score: 0.6946502057613169
Distribution:  0.6625676340383669
Failed to gain:  -0.02754549926217409
********************
********************


 99%|█████████▉| 99/100 [10:10<00:06,  6.10s/it]

Training accuracy on selected features: 1.000
Testing accuracy on selected features: 0.892
F1 Score: 0.8875838926174497
Distribution:  0.5291985501409585
Imp features:  Index(['ATP2ter', 'r1466', 'sink_lnlncacoa[c]', 'FACOAL245_2', 'SPHGNte',
       'URIt', 'sink_lnlc[c]', 'SMS', 'EX_pe_hs[e]', 'r0430'],
      dtype='object')
--------------------
Gain:  0.3628674989931534
--------------------
********************
********************


100%|██████████| 100/100 [10:16<00:00,  6.17s/it]

Training accuracy on selected features: 1.000
Testing accuracy on selected features: 0.828
F1 Score: 0.8621526841111411
Distribution:  0.6200269723533378
Imp features:  Index(['ATP2ter', 'HMGCOAtm', 'FACOAL245_2', 'r0488', 'MEV_Rt', 'r1466',
       'EX_mev_R[e]', 'PEt', 'DTTPtn', 'biomass_reaction'],
      dtype='object')
--------------------
Gain:  0.20768712070128115
--------------------
********************
********************


In [139]:
print('Average Accuracy score: {}'.format(np.mean(accuracy_tracker)))
print('Average F1 score: {}'.format(np.mean(f1_tracker)))
print('Average validation distribution: {}'.format(np.mean(val_distribution_tracker)))
print('Average precisions score: {}'.format(np.mean(precision_tracker)))
print('Average recall score: {}'.format(np.mean(recall_tracker)))

Average Accuracy score: 0.8069434975573351
Average F1 score: 0.8201534111475917
Average validation distribution: 0.5656316648574554
Average precisions score: 0.8561159445851869
Average recall score: 0.7979496544539648


In [146]:
# Top important reactions
average_rank = pd.DataFrame({'rank': pd.concat(imp_df).groupby('reactions')['rank'].mean()}).reset_index()
average_rank = average_rank.sort_values(by='rank').head(20).reset_index(drop=True)
average_rank.head(20)

,reactions,rank
0,ATP2ter,1.596774
1,FACOAL245_2,6.564516
2,sink_lnlncacoa[c],7.500000
3,HMGCOAtm,11.661290
4,MEV_Rt,14.196721
5,EX_mev_R[e],14.508197
6,r0488,15.387097
7,EX_uri[e],16.603448
8,EX_Rtotal[e],17.311475
9,r1466,17.583333


In [145]:
pd.DataFrame(imp_list)[0].value_counts().head(20)

ATP2ter              62
HMGCOAtm             62
FACOAL245_2          62
sink_lnlncacoa[c]    62
r0430                62
r0488                62
CLS_hs               61
DATPtn               61
DNDPt9m              61
EX_mev_R[e]          61
r1004                61
EX_sphgn[e]          61
RTOTALt              61
MEV_Rt               61
EX_Rtotal[e]         61
URIt                 61
r1050                61
EX_pglyc_hs[e]       60
r1466                60
LPS2                 60
Name: 0, dtype: int64